# Detecção de Spam utilizando Aprendizado de Máquina

Alunos:
- CARLOS EDUARDO DOS SANTOS SILVA TERTULIANO
- JOÃO GUILHERME LOPES ALVES DA COSTA
- RAFAEL SILVA FREIRE

Professor: Daniel Sabino Amorim De Araujo

In [ ]:
# Importar bibliotecas
import numpy as np
import pandas as pd
import nltk
import gdown

In [ ]:
# importe do database
url = 'https://drive.google.com/file/d/1R5XkSXKpYB16KBSrvKg1w_-tL43FzVtq/view?usp=sharing'
output = 'email_messages.csv'
gdown.download(url=url, output=output, quiet=False, fuzzy=True)
np.random.seed(5)
email_messages = pd.read_csv('email_messages.csv', on_bad_lines='skip')
print(len(email_messages))

Downloading...
From: https://drive.google.com/uc?id=1R5XkSXKpYB16KBSrvKg1w_-tL43FzVtq
To: /content/email_messages.csv
100%|██████████| 140M/140M [00:04<00:00, 31.9MB/s]


83448


In [ ]:
email_messages_list = dfs_divididos = np.array_split(email_messages, 6)
email_messages = email_messages_list[0]
email_messages.head()

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...


In [ ]:
from nltk.stem.snowball import SnowballStemmer
import re

nltk.download('punkt')

stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):

    # tokeniza por setença e depois por palavra
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]

    # remove caracteres estranhos
    filtered_tokens = [token for token in tokens if re.search('[a-zA-Z]', token)]

    # stemmiza os tokens
    stems = [stemmer.stem(token) for token in filtered_tokens]

    return stems

# importa TfidfVectorizer para criar vetores TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# instancia um objeto TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem,
                                 ngram_range=(1,3))

tfidf_matrix = tfidf_vectorizer.fit_transform([x for x in email_messages["text"].values.astype('U')])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewh

In [ ]:
# Rede Neural
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = tfidf_matrix.toarray()
y = email_messages['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertendo os dados para um formato compatível com TensorFlow
scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Criando um modelo de rede neural usando TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinando o modelo
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test))
y_pred = model.predict(X_test_scaled)
y_pred = (y_pred > 0.5).astype(int)

# Avaliando o modelo
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Acurácia do modelo nos dados de teste: {accuracy:.2f}')

confusion = confusion_matrix(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

print('Matriz de Confusão:')
print(confusion)
print('Relatório de Classificação:')
print(classification_report_str)

Epoch 1/10
348/348 [==============================] - 2s 3ms/step - loss: 0.5670 - accuracy: 0.7107 - val_loss: 0.4978 - val_accuracy: 0.7372
Epoch 2/10
348/348 [==============================] - 1s 3ms/step - loss: 0.4847 - accuracy: 0.7598 - val_loss: 0.4564 - val_accuracy: 0.7840
Epoch 3/10
348/348 [==============================] - 1s 2ms/step - loss: 0.4523 - accuracy: 0.7850 - val_loss: 0.4401 - val_accuracy: 0.7850
Epoch 4/10
348/348 [==============================] - 1s 3ms/step - loss: 0.4384 - accuracy: 0.7939 - val_loss: 0.4330 - val_accuracy: 0.7969
Epoch 5/10
348/348 [==============================] - 1s 3ms/step - loss: 0.4288 - accuracy: 0.7999 - val_loss: 0.4232 - val_accuracy: 0.8012
Epoch 6/10
348/348 [==============================] - 1s 3ms/step - loss: 0.4197 - accuracy: 0.8063 - val_loss: 0.4267 - val_accuracy: 0.7991
Epoch 7/10
348/348 [==============================] - 1s 3ms/step - loss: 0.4126 - accuracy: 0.8113 - val_loss: 0.4201 - val_accuracy: 0.8055
Epoch 

In [ ]:
# Arvore de Decisão
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = tfidf_matrix.toarray()
y = email_messages['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertendo os dados para um formato compatível com a árvore de decisão
scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Criando e treinando o modelo da árvore de decisão
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train_scaled, y_train)

# Fazendo previsões nos dados de teste
y_pred = tree_model.predict(X_test_scaled)

# Calculando e imprimindo a acurácia do modelo nos dados de teste
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo de árvore de decisão nos dados de teste: {accuracy:.2f}')

confusion = confusion_matrix(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

print('Matriz de Confusão:')
print(confusion)
print('Relatório de Classificação:')
print(classification_report_str)

Acurácia do modelo de árvore de decisão nos dados de teste: 0.82
Matriz de Confusão:
[[1025  310]
 [ 196 1251]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.84      0.77      0.80      1335
           1       0.80      0.86      0.83      1447

    accuracy                           0.82      2782
   macro avg       0.82      0.82      0.82      2782
weighted avg       0.82      0.82      0.82      2782



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = tfidf_matrix.toarray()
y = email_messages['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertendo os dados para um formato compatível com o k-NN
scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Criando e treinando o modelo k-NN
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train_scaled, y_train)

# Fazendo previsões nos dados de teste
y_pred = knn_model.predict(X_test_scaled)

# Calculando e imprimindo a acurácia do modelo nos dados de teste
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo k-NN nos dados de teste: {accuracy:.2f}')

confusion = confusion_matrix(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

print('Matriz de Confusão:')
print(confusion)
print('Relatório de Classificação:')
print(classification_report_str)

Acurácia do modelo k-NN nos dados de teste: 0.82
Matriz de Confusão:
[[1047  288]
 [ 217 1230]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.83      0.78      0.81      1335
           1       0.81      0.85      0.83      1447

    accuracy                           0.82      2782
   macro avg       0.82      0.82      0.82      2782
weighted avg       0.82      0.82      0.82      2782

